# 图书推荐教程
### 数据悦读大赛作品
这个笔记本展示了如何对原始数据集进行预处理、特征提取、模型训练、评估等，最终将生成的数据传送至远端服务器上进行推荐。笔记本中可能需要修改的是原始数据的存放路径，即文件中的DataDir

In [1]:
import pandas as pd
ProjectFolderDir = './'   # 项目文件夹的路径是相对于本笔记本路径而言的，当然也可以使用绝对路径
DataDir = './data/raw/' # 原始数据路径，根据实际情况修改

初步分析阶段先将字段以字符串形式录入，便于后期统一格式

In [2]:
BigData = pd.read_csv(DataDir + "浙江大学-图书外借数据-2013~2018.csv", encoding='utf8', dtype=str)       
# 注意~是英文输入，读取不到可以重命名一下

In [3]:
SmallData = pd.read_csv(DataDir + "浙江大学-图书外借数据-2019.csv", encoding='utf8', dtype=str)         

下面是构建模型所需要的数据列

In [4]:
user_columns = ['PATRON_ID', 'STUDENT_GRADE', 'PATRON_DEPT', 'PATRON_TYPE']
# 用户侧特征       用户id          年级            学生学院       学生类型
item_columns = ['ITEM_ID', 'SUBLIBRARY', 'ITEM_CALLNO', 'PUBLISH_YEAR', 'AUTHOR', 'TITLE', 'PRESS']
# 物品侧特征       记录号      馆藏地         图书索书号        出版年          作者       题目    出版社
time = 'LOAN_DATE'

将两份数据集合并

In [5]:
UsedColumns = user_columns + item_columns + [time]
UsedData = pd.concat([BigData[UsedColumns], SmallData[UsedColumns]])

本项目将借阅记录中的空值字段采用相同值’na'进行填充，特征处理时将其作为空特征。对于图书类别，将其索书号按/分割后得到图书大类CALLNO1与小类CALLNO2，将借阅日期转为整形，便于之后比较大小。

In [6]:
import re

def get_Bletter(str0):   # 取出大写字母
    b = re.sub(u"([^\u0041-\u007a])", "", str0)
    return b

UsedData = UsedData.fillna(value='na')
UsedData['CALLNO1'] = UsedData['ITEM_CALLNO'].str.split('/', expand=True)[0].map(lambda x: get_Bletter(str(x)))
UsedData['CALLNO2'] = UsedData['ITEM_CALLNO'].str.split('/', expand=True)[1].map(lambda x: get_Bletter(str(x)))
UsedData[time]=UsedData[time].astype(int)
UsedData = UsedData.drop(columns='ITEM_CALLNO')  # 删掉不再需要的列

此时可以看到UsedData中的空值消失，并且多出了两个类别列

In [7]:
UsedData.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2260649 entries, 0 to 192700
Data columns (total 13 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   PATRON_ID      object
 1   STUDENT_GRADE  object
 2   PATRON_DEPT    object
 3   PATRON_TYPE    object
 4   ITEM_ID        object
 5   SUBLIBRARY     object
 6   PUBLISH_YEAR   object
 7   AUTHOR         object
 8   TITLE          object
 9   PRESS          object
 10  LOAN_DATE      int64 
 11  CALLNO1        object
 12  CALLNO2        object
dtypes: int64(1), object(12)
memory usage: 241.5+ MB


将数据存入./data/processed中，作为后续分析数据

In [8]:
import os


if not os.path.exists(ProjectFolderDir+"data/processed/"):
    os.makedirs(ProjectFolderDir+"data/processed/") 

UsedData.to_csv(ProjectFolderDir+"data/processed/ZJULibrary2013_2019.csv")

In [9]:
del BigData # 将之前的大数据变量清空，空出一部分内存
del SmallData
del UsedData

#### 若已将两个数据集合并可以直接从此处开始
调用代码库中的主函数进行模型训练、评估等操作

In [10]:
import sys
import os
import torch
sys.path.append(ProjectFolderDir)

from scripts.MainFile import main

# 创建缓存目录

if not os.path.exists(ProjectFolderDir+"temp/"):
    os.makedirs(ProjectFolderDir+"temp/")
if not os.path.exists(ProjectFolderDir+"log/"):
    os.makedirs(ProjectFolderDir+"log/")

# 这边可能会报warning:Please update jupyter and ipywidgets，不过对本项目无影响

调用主函数即可，运行的日志文件保存在ProjectFolderDir/log目录下

In [11]:
main(ProjectFolderDir,  # 之前定义的项目路径
      neg_ratio= 3,  # 负采样倍率
      min_item= 5,  # 最短序列要求
      seq_max_len= 20,  # 截断序列长度
      load= False, # 是否从已有数据读取
      batch_size= 1024, # batch大小
      user_params= [512, 512, 256, 128, 64], # 读者塔MLP参数
      item_params= [1024, 512, 256, 128, 64], # 图书塔MLP参数，要确保最后的维度一致
      temperature= 0.02, # 温度系数
      learning_rate= 1e-4, # 学习率
      weight_decay= 1e-4, # 正则化系数
      optimizer_fn= torch.optim.Adam,  # 优化器 
      epoch= 5, # 训练epoch
      topk= 100 #推荐topk个商品 
    )  # 大约需要1小时，Hit Rate 25%  最后的评估可能会耗时较久

Start: 2023-07-25 20:17:08


generate train set, validation set and test set:: 100%|██████████| 118358/118358 [00:28<00:00, 4179.63it/s]


n_train: 6246232, n_val: 1561559, n_test: 867533
40557 cold start user droped 
train set, validation set and test set have saved in data/processed/data_process.npy
standard data has been generated
epoch: 0


calculate auc on validation data: 100%|██████████| 1525/1525 [00:19<00:00, 78.64it/s]


auc on train data: 0.8632906171460298
auc on validation data: 0.8356946873747348
epoch: 1


...

calculate confusion matrix on test data: 100%|██████████| 848/848 [00:11<00:00, 75.30it/s]



confusion_matrix on test data: {'tn': 0.6883668978586406, 'fp': 0.06202415354804947, 'fn': 0.07842814048572215, 'tp': 0.17118080810758785}
auc: 0.8595, recall: 0.6858, precision: 0.7340
generated user embedding and item embedding


item inference: 100%|██████████| 685/685 [00:05<00:00, 126.02it/s]


{'HR': 0.24606367527409673, 'n_hit': 19144, 'n_total': 77801}
embedding has been saved to ./temp/
Finish: 2023-07-25 22:38:29


0

运行结束后，在项目路径{ProjectFolderDir}下会产生文件

{ProjectFolderDir}/data/processed/ZJULibrary2013_2019.csv # 合并整理后的数据集

{ProjectFolderDir}/data/processed/data_process.npy # 训练集，验证集，测试集  

{ProjectFolderDir}/data/processed/item_user.npy # 用于召回时匹配embedding  *

{ProjectFolderDir}/data/processed/raw_id_maps.npy # 原始的id字典    *

{ProjectFolderDir}/log/{Start time}.txt # 以开始训练时间命名的日志文件

{ProjectFolderDir}/temp/item.ann.index # 保存的用于召回的索引文件  *

{ProjectFolderDir}/temp/item_embeddding.pth # 图书的embedding向量  *

{ProjectFolderDir}/temp/user_embedding.pth # 读者的embedding向量  *

{ProjectFolderDir}/temp/model.pth # 训练的模型参数

其中注释后标*的是召回需要的文件（需要上传至服务器）

可以在终端中运行

python ./scripts/RecItem.py '997e765063b98413f5b079c026468f8'

测试推荐